# 얼굴 인식하여 캐릭터 씌우기

## Face Detection vs Face Recognition

패치키 설치
> pip install mediapipe

In [13]:
import cv2
import mediapipe as mp

def overlay(image, x, y,w,h, overlay_image): # 대상 이미지, x,y좌표,  width, height, 덮어씌울 이미지
    alpha = overlay_image[:,:,3]
    mask_image = alpha / 255 # 0 - 255 -> 255로 나누면 0 ~ 1 사이 값(1: 불투명, 0: 완전)
    
    for c in range(0,3):
        image[y-h: y+h, x-w:x+w, c] = ((overlay_image[:, :, c] * mask_image + image[y-h: y+h, x-w:x+w, c]) * (1 - mask_image))

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈 사용


# 동영상 파일 열기
cap = cv2.VideoCapture('face.mp4')

# 이미지 불러오기
image_right = cv2.imread('right.png', cv2.IMREAD_UNCHANGED)
image_left = cv2.imread('left.png', cv2.IMREAD_UNCHANGED)
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)


with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            # If loading a video, use 'break' instead of 'continue'.
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀(귀구슬점, 이주)
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape # height, width, channel
                
                right_eye = (int(right_eye.x * w), int(right_eye.y * h))
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
#                 # 양 눈에 동그라미 그리기
#                 cv2.circle(image, right_eye, 50,(255,0,0), 10, cv2.LINE_AA)
#                 cv2.circle(image, left_eye, 50,(0,255,0), 10, cv2.LINE_AA)
#                 # 코에 동그라미 그리기
#                 cv2.circle(image, nose_tip, 50,(0,255,255), 10, cv2.LINE_AA)
#                 image[right_eye[1]-50:right_eye[1]+50,right_eye[0]-50:right_eye[0]+50] =image_right
#                 image[left_eye[1]-50:left_eye[1]+50,left_eye[0]-50:left_eye[0]+50] =image_left
#                 image[nose_eye[1]-50:nose_eye[1]+50,nose_eye[0]-50:nose_eye[0]+50] =image_nose

                overlay(image, *right_eye, 50, 50, image_right)
                overlay(image, *left_eye, 50, 50, image_left)
                overlay(image, *nose_tip, 150, 50, image_nose)
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy = 0.5))
        if cv2.waitKey(1)  == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [12]:
image_right = cv2.imread('right.png', cv2.IMREAD_UNCHANGED)
image_right.shape

(100, 100, 4)